# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read tables

flights = pd.read_csv('./data/flights.csv')
# Load flights aggregate data
flight_delay_aggregate_mth = pd.read_csv('./data/flight_delay_aggregate_monthly.csv')
flight_delay_aggregate_dow = pd.read_csv('./data/flight_delay_aggregate_day_of_week.csv')
flight_delay_aggregate_arrive_hour= pd.read_csv('./data/flight_delay_aggregate_arrive_hour.csv')
flight_airport_traffic = pd.read_csv('./data/flight_airport_traffic.csv')

# Load passengers aggregate data
passengers_flight_montly_aggregate = pd.read_csv('./data/passengers_flight_montly_aggregate.csv')
passengers_carrier_monthly_aggregate = pd.read_csv('./data/passengers_carrier_monthly_aggregate.csv')
passengers_airport_monthly_aggregate= pd.read_csv('./data/passengers_airport_monthly_aggregate.csv')

# Load fuel comsumption aggregate data
fuel_comsumption_monthyl_aggregate= pd.read_csv('./data/fuel_comsumption_monthyl_aggregate.csv')

In [3]:
flights.shape

(9808, 28)

In [4]:
# Take target variable out of flights data set
y = flights['arr_delay']
flights = flights.drop('arr_delay', axis=1)
# join tables

In [5]:
flights.shape

(9808, 27)

In [6]:
flights_train.shape

NameError: name 'flights_train' is not defined

In [7]:
flights_train = pd.merge(flights, flight_delay_aggregate_mth, how='left', on=['mkt_unique_carrier', 'origin_airport_id', 'dest_airport_id',  'month'])


flights_train = pd.merge(flights_train, flight_delay_aggregate_dow, how='left', on=['mkt_unique_carrier', 'origin_airport_id', 'dest_airport_id',  'day_of_week'])
# Join Flight delays aggregate
flights_train = pd.merge(flights_train, flight_delay_aggregate_arrive_hour, how='left', on=['mkt_unique_carrier', 'origin_airport_id', 'dest_airport_id', 'crs_arr_hour'])

# Join Airport traffic
orig = flight_airport_traffic[['airport_id','month', 'total_flights']]
orig.columns = ['origin_airport_id','month','origin_total_flights']
dest = flight_airport_traffic[['airport_id','month','total_flights']]
dest.columns = ['dest_airport_id','month','dest_total_flights']

flights_train = pd.merge(flights_train, orig, how='left', on=['origin_airport_id','month'])
flights_train = pd.merge(flights_train, dest, how='left', on=['dest_airport_id','month'])



flights_train = pd.merge(flights_train, passengers_flight_montly_aggregate, how='left', left_on=['mkt_unique_carrier', 'origin_airport_id', 'dest_airport_id','month'], right_on=['unique_carrier', 'origin_airport_id', 'dest_airport_id','month'])

flights_train = pd.merge(flights_train, passengers_carrier_monthly_aggregate, how='left', left_on=['mkt_unique_carrier', 'month'], right_on=['unique_carrier', 'month'])





flights_train = pd.merge(flights_train, fuel_comsumption_monthyl_aggregate, how='left', on=['mkt_unique_carrier', 'month'])

orig_pass = passengers_airport_monthly_aggregate[['airport_id','month', 'airport_month_flight_seats', 'airport_month_passengers']]
orig_pass.columns = ['origin_airport_id','month', 'orig_airport_month_flight_seats', 'orig_airport_month_passengers']
dest_pass = passengers_airport_monthly_aggregate[['airport_id','month', 'airport_month_flight_seats', 'airport_month_passengers']]
dest_pass.columns = ['dest_airport_id','month', 'dest_airport_month_flight_seats', 'dest_airport_month_passengers']

flights_train = pd.merge(flights_train, orig_pass, how='left', on=['origin_airport_id','month'])
flights_train = pd.merge(flights_train, dest_pass, how='left', on=['dest_airport_id','month'])

In [8]:
flights_train.shape

(9808, 65)

In [9]:
flights_train.to_csv('./data/train.csv', index=False)
y.to_csv('./data/target.csv', index=False)

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.